### This notebook checks the QE curves as they are in obs_lsst_data repo

This is a modified version of the notebook by Simon Krughoff
https://github.com/lsst-sitcom/notebooks/blob/master/contrib/get_QE_curve.ipynb

Simon's notebook looks at all the QE curves that are in the ts8/CALIB. 
Here we specify a raft name (e.g. R22), then only plot the 9 CCDs for that raft.

Other added features:
* compare to the design curve in syseng_throughput, and 
* plot the raw data points so that we can examine the interpolation/extrapolation. 
 - We don't want to make the plots too busy so we only show raw data points averaged over the amps.

In [1]:
# make sure this is w_2019_49 or later
! eups list | grep lsst_distrib 

In [2]:
from bokeh.plotting import figure, show
from bokeh.layouts import gridplot
from bokeh.io import output_notebook, export_png
output_notebook()

from lsst.daf.persistence import Butler, NoResults
import os, numpy
import pandas as pd
from astropy import units as u
from scipy.interpolate import interp1d

Loading BokehJS ...

In [3]:
import syseng_throughputs as st
from lsst.geom import Point2D, Point2I
from lsst.afw.cameraGeom import FIELD_ANGLE, PIXELS
from lsst.obs.lsst.lsstCamMapper import LsstCamMapper

In [16]:
rname = 'R10'
#rname = 'R22'
#rname = 'R01'
#rname = 'R11'
#rname = 'R20'
#rname = 'R21'
#rname = 'R30'
#rname = 'R12'
#rname = 'R02'
#rname = 'R31'
#rname = 'R03'
#rname = 'R34'
#rname = 'R13'
#rname = 'R23'
#rname = 'R14'
#rname = 'R32'
#rname = 'R24'
#rname = 'R41'
#rname = 'R42'
#rname = 'R33'
#rname = 'R43'

In [17]:
#DATADIR = f"{os.environ['OBS_LSST_DIR']}/ts8/CALIB"
DATADIR = f"{os.environ['OBS_LSST_DIR']}/lsstcam/CALIB" #we can use this after DM-22605 gets merged
print(DATADIR)
butler = Butler(DATADIR)
cam = butler.get('camera')

/Users/lynnej/lsst_stack/stack/miniconda3-py38_4.9.2-0.7.0/DarwinX86/obs_lsst/22.0.1-61-gbd5239c+96886d66ff/lsstcam/CALIB
lsst.LsstCamMapper WARN: Unable to find valid calib root directory
lsst.CameraMapper INFO: Loading Posix exposure registry from /Users/lynnej/lsst_stack/stack/miniconda3-py38_4.9.2-0.7.0/DarwinX86/obs_lsst/22.0.1-61-gbd5239c+96886d66ff/lsstcam/CALIB
lsst.CameraMapper INFO: Loading calib registry from /Users/lynnej/lsst_stack/stack/miniconda3-py38_4.9.2-0.7.0/DarwinX86/obs_lsst/22.0.1-61-gbd5239c+96886d66ff/lsstcam/CALIB/calibRegistry.sqlite3


In [18]:
# mapping based on 
#https://confluence.slac.stanford.edu/pages/viewpage.action?spaceKey=LSSTCAM&title=Raft+Delivery+and+Acceptance+Testing+Status
dd = pd.read_csv('raftInstall.csv',index_col=0)
rtmname = dd.rtm.loc[rname]
vendor = dd.vendor[rname]
print(rname, rtmname,vendor)

R10 RTM-023 ITL


### Retrieve the vendor design curve (e2v or ITL)

In [19]:
defaultDirs = st.setDefaultDirs()
vendorDir = defaultDirs['detector']+'/../'+vendor.lower()
addLosses = False 
detector = st.buildDetector(vendorDir, addLosses)

#### if there are only 6 measurements, we will have to interpolate beyond [min, max] 
The way this is done in syseng_throughput is to set fill_value = 0. This looks OK when we have >30 measurements.
For the 6-point curves, if we do that, the interpolated curve will be artificially low at the high end and low end.
We could compile all the other curves from other rafts but the same vendor, average them, then fit to the 6 points.
But that makes it too complicated. Merlin and Robert actually think these data are not and will not be useful. Why bother?
#### Instead, the simplest solution, is to get the two points where the vendor design curve goes down to zero, add those two points to our 6 points.

In [20]:
import numpy as np
idx = np.where(detector.sb>0.01)
idx1=idx[0][0]-1
idx2=idx[0][-1]+1

x1 = detector.wavelen[idx1]
y1 = detector.sb[idx1]
x2 = detector.wavelen[idx2]
y2 = detector.sb[idx2]

In [21]:
def make_subplot(plot, n):

    interp_list = []
    interp_wlen = detector.wavelen * u.nm
    for k in qe_curve.data:
        if len(qe_curve.data[k][0])>10:
            wlen = qe_curve.data[k][0]
            eff = qe_curve.data[k][1]
            
        else:
            aa = np.append(x1, qe_curve.data[k][0].value)
            aa = np.append(aa, x2)
            wlen = aa * qe_curve.data[k][0].unit

            aa = np.append(y1, qe_curve.data[k][1].value)
            aa = np.append(aa, y2)
            eff = aa * qe_curve.data[k][1].unit
            
        #print(np.max(eff))
        if np.max(eff.value)>150:
            print('These seem too LARGE ', k)
            print(eff)
            eff[np.where(eff.value>100)] = 0
            
        if len(qe_curve.data[k][0])>10:
            f = interp1d(wlen.value, eff.value, fill_value=0, bounds_error=False, kind='quadratic')
        else:
            f = interp1d(wlen.value, eff.value, fill_value=0, bounds_error=False, kind='slinear')#quadratic causes overshoot
            
        eff[np.isnan(eff)] = 0
        plot.circle(wlen.value, eff.value, line_color='black', line_width=3, legend_label='Amp data')
        
        interp_eff = f(interp_wlen.value)*eff.unit
        #alternatively we could do (only for >10 QE measurements)
        #amp_point = det[k].getBBox().getCenter()
        #interp_eff = qe_curve.evaluate(det, amp_point, interp_wlen, kind='quadratic')
        
        interp_list.append(interp_eff)
        
    interp_arr = numpy.array([arr for arr in interp_list])
    median_eff = numpy.median(interp_arr, axis=0)
    #print(interp_arr)
    #print(median_eff)
    plot.line(interp_wlen, median_eff, color='black', line_width=2, legend_label = 'Amp median')
    plot.line(detector.wavelen, detector.sb*100, color='red', line_width=2, legend_label = '%s design'%vendor)
    plot.legend.location = "center"
    plot.legend.click_policy="hide"
    
    if n > 6:
        plot.xaxis.axis_label = 'wavelength (nm)'
        
    if n in (1, 4, 7):
        plot.yaxis.axis_label = 'efficiency (%)'
        
    #plot.text(x=[320,], y=[20,], text=[f'{det.getName().replace(rtmname, rname)}',])
    plot.text(x=[320,], y=[20,], text=[f'{det.getName()}',]) #use this line after DM-22605 gets merged
    plot.text(x=[320,], y=[10,], text=[f'{rtmname}',])

In [22]:
plots = []
for det in cam:
    rname1, dname = det.getName().split('_')
    #if rname1 != rtmname:
    if rname1 != rname: #use this line after DM-22605 gets merged
        continue;

    try:
        #qe_curve = butler.get('qe_curve', raftName=rtmname, detectorName=dname, taiObs='2000-01-01T00:00:00')
        #qe_curve = butler.get('qe_curve', raftName=rname, detectorName=dname, taiObs='2000-01-01T00:00:00') #DM-22605 gets merged
        qe_curve = butler.get('qe_curve', raftName=rname, detectorName=dname, calibDate='1970-01-01T00:00:00') 
    except NoResults:
        print('No results found for this detector', det.getName())
        continue  # 

    if len(plots) > 0:
        plots.append(figure(x_range=plots[0].x_range, y_range=plots[0].y_range))
    else:
        plots.append(figure())
    print(rname1, rname)
    make_subplot(plots[-1], len(plots))

R10 R10
R10 R10
R10 R10
R10 R10
R10 R10
R10 R10
R10 R10
R10 R10
R10 R10


In [23]:
grid = gridplot(plots, ncols=3, plot_width=350, plot_height=200)
show(grid)

In [12]:
pltDir = os.path.join('dm_plots', rname)
if not os.path.exists(pltDir):
    os.mkdir(pltDir)
pngPath = os.path.join(pltDir, 'QE_%s.png'%rname)
export_png(grid, filename=pngPath)

RuntimeError: Neither firefox and geckodriver nor a variant of chromium browser and chromedriver are available on system PATH. You can install the former with 'conda install -c conda-forge firefox geckodriver'.

In [13]:
from lsst.obs.lsst.ts8 import Ts8Mapper
cam = Ts8Mapper().camera

/var/folders/tr/bjzqy0h17kx31fhndw6050d80000gn/T/ipykernel_11401/4187161171.py:2: FutureWarning: Gen2 Butler has been deprecated (Ts8Mapper). It will be removed sometime after v23.0 but no earlier than the end of 2021.
  cam = Ts8Mapper().camera


lsst.CameraMapper INFO: Loading Posix exposure registry from .


In [14]:
cam['RTM-024_S01'].getSerial()

'E2V-CCD250-372'

In [15]:
cam['RTM-009_S00']